# Python names and variables

Reference: Ned Batchelder's blog [post](https://nedbatchelder.com/text/names.html)

I think of Python variable names in terms of pointers to values stored in memory. However, if so, I would expect...

In [2]:
x = 23
y = x
x = 12
y

23

...`y=12`  instead of `y=23`. The reason is that when `x = 12` Python **rebinds** pointer *x* to a new values. Instead of the word *pointer* people use *reference* or *name*. Note that if a certain value has no names referring to it, it is cleaned up by the garbage collector. Given the above snippet, the following one might feel confusing at first:

In [4]:
x = [1, 2, 3]
y = x
x.append(4)
y

[1, 2, 3, 4]

In this case, `x.append(4)` does not reassign the name `x` to a new location but alters the corresponding value. In other words, it performes a change *in-place*. There are two types of variables in Python:
- **immutables** I can't change their value. Numbers, strings and tuples are immutable. I can, however, reassign their reference (**rebind**) to some other values.
- **mutable**: I can changes their values in-place, usually using methods. *i.e.*, lists, dicts, user-defined objects are mutable. Clearly, I can also rebind their names to different values using assignments.
